In [ ]:
# Headers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import ast

In [ ]:
# Imports the dataset from a csv file
dataset = pd.read_csv("train.csv")

In [ ]:
# Plots some of the dataset's information
dataset.describe()

In [ ]:
# Drops unwanted columns
dataset = dataset.drop(columns=['tr_attempts'])
dataset = dataset.drop(columns=['tr_id'])

In [ ]:
# Removes observations with empty data
dataset.dropna(inplace=True)

In [ ]:
# Cuts the dataset to a fraction of its size
def cut_dataset(dataset, factor=1):
	dataset_temp = dataset.drop_duplicates(keep='first')
	return dataset_temp.sample(frac=factor)

In [ ]:
# Balances the dataset by cutting the larger class
def balance_dataset(dataset, target_name, factor=1.0):
	dataset_temp = dataset.drop_duplicates(keep='first')
	class_0 = dataset_temp[dataset_temp[target_name] == 0]
	class_1 = dataset_temp[dataset_temp[target_name] == 1]
	
	if len(class_0) > len(class_1):
		class_1 = class_1.sample(int(len(class_1) * factor))
		class_0 = class_0.sample(len(class_1))
	else:
		class_0 = class_0.sample(int(len(class_0) * factor))
		class_1 = class_1.sample(len(class_0))

	return pd.concat([class_0, class_1])

In [ ]:
# Cuts pairs
def cut_pairs(dataset, factor=1.0):
    pairs = dataset[['tr_src', 'tr_dst']].drop_duplicates()
    sampled_pairs = pairs.sample(frac=factor)
    display(len(sampled_pairs))
    filtered_dataset = dataset.merge(sampled_pairs, on=['tr_src', 'tr_dst'], how='inner')
    return filtered_dataset

In [ ]:
balanced_dataset = balance_dataset(dataset, 'route_changed', factor=0.001)
# balanced_dataset = cut_dataset(dataset, factor=0.001)
# balanced_dataset = cut_pairs(dataset, 0.005)
balanced_dataset.describe()

In [ ]:
# Parses the all_rtts column from string to list
def parse_rtts(stringzinha):
    try:
        return ast.literal_eval(stringzinha)
    except:
        return []

# Calculates mean and std of all_rtts
rtt_lists = balanced_dataset["all_rtts"].apply(parse_rtts)
balanced_dataset["mean_rtt"] = rtt_lists.apply(lambda x: np.mean(x) if len(x) > 0 else 0.0)
balanced_dataset["std_rtt"] = rtt_lists.apply(
	lambda x: np.std(x, ddof=1) if len(x) > 1 else 0.0
)

# Drops the all_rtts column
balanced_dataset = balanced_dataset.drop(columns=['all_rtts'])

In [ ]:
balanced_dataset.describe()

In [ ]:
# Pearson correlation
cor_pearson = balanced_dataset.corr(method='pearson')
plt.figure(figsize=(7, 7))
sbn.heatmap(cor_pearson, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Spearman correlation
cor_spearman = balanced_dataset.corr('spearman')
plt.figure(figsize=(7, 7))
sbn.heatmap(cor_spearman, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Drops the total_replies_last_hop column
balanced_dataset = balanced_dataset.drop(columns=['total_replies_last_hop'])

In [ ]:
# Exports the treated dataset to a csv file
balanced_dataset.to_csv("treated_train.csv", index=False)